In [20]:
import os
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import torch.cuda as cuda
from torch import optim
from torch.utils.data import DataLoader, Dataset

import time

In [21]:
# Hyper Parameters
BATCH = 64
EPOCHS = 40
# INPUT_SIZE = 6
# LR = 0.01
# d_model = 512
# heads = 8
# HIDDEN_SIZE = 32
# h_state = None
TIME_STEP = 12
STEPS = 1
DEVICE = torch.device('cuda' if cuda.is_available() else 'cpu')
option = 'HTQR'  # MFCC
learning_rate = 0.001

In [22]:
class TimbreData(Dataset):
    def __init__(self): 
        x = torch.from_numpy(np.reshape(train_x,newshape=(-1,25)))  # shape: 25x 25
        y = torch.from_numpy(np.reshape(train_y,newshape=(-1,1)))  # shape:  
        self.trainx = x.view(-1, 1, 25, x.shape[1]).float()
        self.trainy = y.view(-1, 1, 25, y.shape[1]).float()
#         testx.shape, testy.shape

        self.len = self.trainy.shape[0]
                                                                      
    def __getitem__(self, index):
        return self.trainx[index], self.trainy[index]

    def __len__(self):
        return self.len

In [23]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 6, 3, stride=1, padding=1), 
            nn.MaxPool2d(2, 2), 
            nn.Conv2d(6, 16, 5, stride=1, padding=0), 
            nn.MaxPool2d(2, 2) ) 
        self.fc = nn.Sequential( 
            nn.Linear(400, 120), 
            nn.Linear(120, 84), 
            nn.Linear(84, 10) ) 

        #         self.hidden = torch.nn.Linear(25, 5)   # hidden layer
#         self.out = torch.nn.Linear(5, num_classes) # output layer
# #         self.features = nn.Sequential(
#             nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=3, stride=2),
#             nn.Conv2d(64, 192, kernel_size=5, padding=2),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=3, stride=2),
#             nn.Conv2d(192, 384, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(384, 256, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(256, 256, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=3, stride=2),
#         )
#         self.classifier = nn.Sequential(
#             nn.Dropout(),
#             nn.Linear(256 * 1 * 1, 4096),
#             nn.ReLU(inplace=True),
#             nn.Dropout(),
#             nn.Linear(4096, 4096),
#             nn.ReLU(inplace=True),
#             nn.Linear(4096, num_classes),
#        )

    def forward(self, x): 
        out = self.conv(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
#         x = F.relu(F.max_pool2d(self.conv1(x),2)) #使用 max 运算执行特定区域的下采样（通常 2x2 像素）；
#         x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2))
#         x = x.view(-1, 320)
#         x = F.relu(self.fc1(x))  #修正线性单元函数，使用逐元素的激活函数 max(0,x)；
#         x = F.dropout(x, training=self.training) #Dropout2D随机将输入张量的所有通道设为零。当特征图具备强相关时，dropout2D 提升特征图之间的独立性；
#         x = self.fc2(x)
#         return F.log_softmax(x, dim=1)  #将 Log(Softmax(x)) 函数应用到 n 维输入张量，以使输出在 0 到 1 之间。

#         x = self.features(x)
#         x = x.view(x.size(0), 256 * 1 * 1)
#         x = self.classifier(x)
#         return x   

In [24]:
def prepare_data(data,label):
  ## separate train and test set
    print("data shape",data.shape)
#     train_data = data.iloc[0:int(0.8*len(data))].as_matrix()
#     test_data = data.iloc[int(0.8*len(data)):].as_matrix()  # ,:,2:
    train_data = data.iloc[0:int(0.8*len(data))].values
    test_data = data.iloc[int(0.8*len(data)):].values  # ,:,2:
    
#     print(len(train_data))
#     print(len(test_data))
    # print("ok data")
    ## get labels
    if label == 0:
        ytrain_data = np.zeros((len(train_data),1), dtype=int)
        ytest_data = np.zeros((len(test_data),1), dtype=int)
    else:
        ytrain_data = label*np.ones((len(train_data),1), dtype=int)
        ytest_data = label*np.ones((len(test_data),1), dtype=int)
    # print(len(ytrain_data))
    # print(len(ytest_data))
    # print("ok label")

    return train_data, ytrain_data, test_data, ytest_data


In [25]:
## read data -> numpy format
data_bad = pd.read_csv('data/{}/{}3120bad.csv'.format(option,option), low_memory=False)
data_mid = pd.read_csv('data/{}/{}3120mid.csv'.format(option,option), low_memory=False)
data_good = pd.read_csv('data/{}/{}3120good.csv'.format(option,option), low_memory=False)

train_bad, ytrain_bad, test_bad, ytest_bad = prepare_data(data_bad,0)
train_mid, ytrain_mid, test_mid, ytest_mid = prepare_data(data_mid,1)
train_good, ytrain_good, test_good, ytest_good = prepare_data(data_good,2)

train_x = np.concatenate((train_bad, train_mid, train_good))[:,2:].astype(float)
train_y = np.concatenate((ytrain_bad, ytrain_mid, ytrain_good))
test_x = np.concatenate((test_bad, test_mid, test_good))[:,2:].astype(float)
test_y = np.concatenate((ytest_bad, ytest_mid, ytest_good))

print(test_x.dtype)
print(test_y.dtype)
print("train shape:", train_x.shape, train_y.shape)                 
print("test shape:", test_x.shape, test_y.shape)

data shape (110253, 27)
data shape (168114, 27)
data shape (179384, 27)
float64
int64
train shape: (366200, 25) (366200, 1)
test shape: (91551, 25) (91551, 1)


In [26]:
testx = torch.from_numpy(np.reshape(test_x, newshape=(-1,25)))
testy = torch.from_numpy(np.reshape(test_y,newshape=(-1,1)))

testx = testx[:-1].view(-1, 1, 25, testx.shape[1]).float()
testy = testy[:-1].view(-1, 1, 25, testy.shape[1]).float()
testx.shape, testy.shape

(torch.Size([3662, 1, 25, 25]), torch.Size([3662, 1, 25, 1]))

In [27]:
testx = testx.to(DEVICE)
testy = testy.to(DEVICE) ###?

trainset = TimbreData()
trainloader = DataLoader(dataset=trainset, batch_size=BATCH,
                         shuffle=True, num_workers=0)

print('Read data: finished')

Read data: finished


In [19]:
model = Model().to(DEVICE)
print(model)
criterian = nn.CrossEntropyLoss(size_average=False)
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
# train 
for i in range(EPOCHS): 
    since = time.time() 
    running_loss = 0. 
    running_acc = 0. 
    for i, (trainx, trainy) in enumerate(trainloader):
        pred = model(trainx.to(DEVICE))
        loss = criterion(pred.squeeze(), trainy.squeeze().to(DEVICE))
        optimizer.zero_grad() 
        # backward 
        loss.backward() 
        optimizer.step() 
        running_loss += loss.data[0] 
        _, predict = torch.max(output, 1) 
        correct_num = (pred == trainy.to(DEVICE)).sum() 
        running_acc += correct_num.data[0] 
        
    running_loss /= len(trainset) 
    running_acc /= len(trainset) 
    print("[%d/%d] Loss: %.5f, Acc: %.2f, Time: %.1f s" %(i+1, epoches, running_loss, 100*running_acc, time.time()-since)) 


Model(
  (conv): Sequential(
    (0): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): Linear(in_features=120, out_features=84, bias=True)
    (2): Linear(in_features=84, out_features=10, bias=True)
  )
)


RuntimeError: Input type (torch.cuda.DoubleTensor) and weight type (torch.cuda.FloatTensor) should be the same

In [9]:
model = Model().to(DEVICE)
print(model)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
for step in range(EPOCHS):
#     scheduler.step()
#     total=0
#     running_loss=0.0
#     running_correct=0
#     print("epoch {}/{}".format(step,EPOCHS))
    for i, (trainx, trainy) in enumerate(trainloader):
        pred = model(trainx.to(DEVICE))
        loss = criterion(pred.squeeze(), trainy.squeeze().to(DEVICE))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                EPOCHS, i * len(trainx.to(DEVICE)), len(train_loader.dataset),
                100. * i / len(train_loader), loss.item()))

#     rmse_train = torch.sqrt(loss)
#     y_ = model(test_x.to(DEVICE))
#     rmse_test = criterion(y_.squeeze(), test_y.squeeze().to(DEVICE))
#     rmse_test = torch.sqrt(rmse_test)
#     loss_test = torch.mean(torch.abs(1 - y_.squeeze() / test_y.squeeze().to(DEVICE)))
#     print('Epoch:%3d' % (step + 1),
#           '| Train RMSE: %.5f' % loss_train,
#           '| Test RMSE : %.5f' % loss_test)

RuntimeError: Expected 4-dimensional input for 4-dimensional weight 10 1 5 5, but got 3-dimensional input of size [1, 25, 25] instead